# Finding area of thymus regions for histocytometry

1. Load in image

2. Calculate area of 3 tissue regions in pixels

3. Convert pixel area to interpertable area

4. Save output as DataFrame w/ sample info and area in both forms

In [ ]:
import pandas as pd 
import numpy as np
import tifffile
import os 
import re

## Functions

In [ ]:
def calculate_tissue_area(img_dir, img_name, XY_resolution= 0.479):
    img= tifffile.imread(os.path.join(img_dir, img_name))

    region_count_table = np.unique(img, return_counts= True)
    total_area = np.sum(img > 0)
    region_prop_table = region_count_table[1]/total_area
    region_areas = region_count_table[1] * (XY_resolution ** 2) ## pixel XY_resolution

    data = {"region_id"          : region_count_table[0],
            "region_pixel_count" : region_count_table[1], 
            "region_area"        : region_areas, 
            "region_prop_table"  : region_prop_table,
            "total_area"         : total_area, 
            "sample"             : img_dir}
    region_df = pd.DataFrame(data)

    return(region_df)

## Running images

In [ ]:
raw_dir = "/stor/scratch/Ehrlich/Users/John/histocytometry/raw_images/images_2023-08-10"
img_dirs= [os.path.join(raw_dir, img_dir) for img_dir in os.listdir(raw_dir) if re.search(r"_[A-D]$", img_dir) and not re.search("Sirpa_C$|CD207_CD11c_XCR1_C$|MerTK_C$", img_dir)]

In [ ]:
df_list = []
for img_dir in img_dirs:
    region_df = calculate_tissue_area(img_dir  = img_dir, 
                                      img_name = "thymus_regions_50um.tif")
    df_list.append(region_df)
    print(f"Done with {img_dir}")

In [ ]:
combined_df = pd.concat(df_list)
print(combined_df.shape)
combined_df.head()

In [ ]:
combined_df.to_csv("/stor/scratch/Ehrlich/Users/John/histocytometry/raw_images/images_2023-08-10/data/region_areas_50um.csv", index= False)